# Optimización de Límites de Crédito

## Exploración de Datos

### 1. - CARGA DE LIBRERIAS

In [27]:
import os
import pandas as pd

In [28]:
print(os.getcwd())


c:\Users\raymar\Documents\JAIRO\CURSOS\Bootcamp Azure Ciencia de Datos\jj-project\notebooks


### 2. -Carga de los Datos

In [29]:
file_path_training = os.path.abspath('../data/train-data/data_training.csv')
file_path_test = os.path.abspath('../data/test-data/data_training.csv')


In [30]:
# read training and test dataset
df_training = pd.read_csv('../data/train-data/data_training.csv', sep=';', index_col='ID', na_values='#N/D')
df_test = pd.read_csv('../data/test-data/data_test.csv', sep=';', index_col='ID', na_values='#N/D')

# display the first few rows of the training dataset
df_training.head()

,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
ID,,,,,,,,,,,,,,,,,,,
5008804,1,1,1,1,0,0,0,0,2,15,427500.0,3.286.857.361.889.700,12.435.573.625.741.800,Working,Higher education,Civil marriage,Rented apartment,Other,1
5008806,1,1,1,0,0,0,0,0,2,29,112500.0,5.879.381.506.807.120,31.047.865.459.249.600,Working,Secondary / secondary special,Married,House / apartment,Security staff,0
5008808,0,0,1,0,1,1,0,0,1,4,270000.0,5.232.140.290.355.030,8.353.354.278.321.930,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0
5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504.343.004.989.800,0.0,Pensioner,Higher education,Separated,House / apartment,Other,0
5008815,1,1,1,1,1,1,0,0,2,5,270000.0,461.939.670.219.101,21.054.504.883.741.600,Working,Higher education,Married,House / apartment,Accountants,0


In [31]:
# Analizar todas las columnas
for column in df_training.columns:
  print(f"Column name: {column}")
  print(f"Data type: {df_training[column].dtype}")
  print(f"Number of unique values: {df_training[column].nunique()}")
  print(f"Number of missing values: {df_training[column].isnull().sum()}")
  print(f"Sample values: {df_training[column].head(3)}")
  print("-" * 60)

Column name: Gender
Data type: int64
Number of unique values: 2
Number of missing values: 0
Sample values: ID
5008804    1
5008806    1
5008808    0
Name: Gender, dtype: int64
------------------------------------------------------------
Column name: Own_car
Data type: int64
Number of unique values: 2
Number of missing values: 0
Sample values: ID
5008804    1
5008806    1
5008808    0
Name: Own_car, dtype: int64
------------------------------------------------------------
Column name: Own_property
Data type: int64
Number of unique values: 2
Number of missing values: 0
Sample values: ID
5008804    1
5008806    1
5008808    1
Name: Own_property, dtype: int64
------------------------------------------------------------
Column name: Work_phone
Data type: int64
Number of unique values: 2
Number of missing values: 0
Sample values: ID
5008804    1
5008806    0
5008808    0
Name: Work_phone, dtype: int64
------------------------------------------------------------
Column name: Phone
Data type: 

In [32]:
df_training.drop(columns = ['Work_phone','Phone','Email','Age','Years_employed'], inplace = True)
df_test.drop(columns = ['Work_phone','Phone','Email','Age','Years_employed'], inplace = True)

In [33]:
#Realizamos un análisis de nulos para validar si existen datos faltantes
df_training.isna().sum().sort_values(ascending = False)
df_test.isna().sum().sort_values(ascending = False)

Gender             0
Own_car            0
Own_property       0
Unemployed         0
Num_children       0
Num_family         0
Account_length     0
Total_income       0
Income_type        0
Education_type     0
Family_status      0
Housing_type       0
Occupation_type    0
Target             0
dtype: int64

In [34]:
import pyarrow as pa
import pyarrow.parquet as pq

In [35]:
# convert data to table
table_training = pa.Table.from_pandas(df_training)
table_test = pa.Table.from_pandas(df_test)

In [36]:
# write tables out to parquet
pq.write_table(table_training, "../data/train-data/data-training.parquet", version="1.0")
pq.write_table(table_test, "../data/test-data/data-test.parquet", version="1.0")

Connect to your workspace

In [38]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)